<a href="https://colab.research.google.com/github/dhdbsrlw/LLM-Implementation-prac/blob/main/T5_Fine_tuning_with_PyTorch_from_JSY_GPU_12G.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentencepiece # (사전 토큰화 없이) 내부 단어 분리
!pip install transformers
!pip install rich[jupyter]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.5 MB/s eta 0:00:00


The command !pip install rich[jupyter] is used in Python to install the "rich" library along with its optional Jupyter notebook integration.

The "rich" library is a Python package that provides enhanced console output and formatting capabilities. It allows you to create more visually appealing and informative text-based user interfaces in the terminal or console. It supports features like syntax highlighting, tables, progress bars, and more.

The [jupyter] part of the command specifies that you want to install the Jupyter notebook integration for the "rich" library. Jupyter notebooks are a popular interactive computing environment that allows you to create and share documents containing code, visualizations, and explanatory text.

By running the !pip install rich[jupyter] command in a Jupyter notebook or a terminal, you are instructing the Python package manager (pip) to download and install the "rich" library along with the necessary dependencies for Jupyter notebook integration. Once installed, you can use the "rich" library in your Python code within Jupyter notebooks to enhance the visual output.

In [2]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Shivanandroy/T5-Finetuning-PyTorch/main/data/news_summary.csv")

In [3]:
df.sample(10)

,headlines,text
15314,I was not present on sets: Ibrahim on Tanushr...,"Muzammil Ibrahim, while commenting on the sexu..."
89248,Hospitals across UK hit by large-scale cyber a...,UK's National Health Service confirmed on Frid...
78629,Players slide on waterlogged pitch as match ge...,English football club QPR and German side Unio...
13956,Amrapali Group seals 9 properties after Suprem...,Amrapali Group on Thursday informed the Suprem...
57623,Lights of Jerusalem mosque turned off over Tru...,The lights illuminating East Jerusalem's al-Aq...
92715,Babri demolition more serious than Gandhi's ki...,AIMIM chief Asaduddin Owaisi has slammed the g...
57435,"Truck spills vodka on US road, affects traffic...",A truck carrying vodka overturned in the Unite...
5444,Madras HC dismisses defamation complaint again...,Madras High Court dismissed criminal defamatio...
37464,Facebook to include autoplay video ads in Mess...,Facebook is planning to include autoplay video...
84095,40-yr-old computer built by Steve Jobs sells f...,An original and working Apple-1 computer built...


In [4]:
df["text"] = "summarize: "+df["text"]

In [5]:
df.head()

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"summarize: Saurav Kant, an alumnus of upGrad a..."
1,Delhi techie wins free food from Swiggy for on...,summarize: Kunal Shah's credit card bill payme...
2,New Zealand end Rohit Sharma-led India's 12-ma...,summarize: New Zealand defeated India by 8 wic...
3,Aegon life iTerm insurance plan helps customer...,summarize: With Aegon Life iTerm Insurance pla...
4,"Have known Hirani for yrs, what if MeToo claim...",summarize: Speaking about the sexual harassmen...


In [6]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

# 윤진) rich 모듈은 터미널 출력을 예쁘게 만들어주는 용도
from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ),
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"),
                        title="Training Status",pad_edge=False, box=box.ASCII)


**(1) torch.nn.functional:** The torch.nn.functional module, often abbreviated as F, is a sub-module of torch.nn that provides a collection of functions commonly used in neural network operations. It includes various activation functions (such as ReLU, sigmoid, tanh), loss functions (such as cross-entropy, mean squared error), and other utility functions. (활성화 함수 등, 신경망 학습에 필요한 함수 정의한다.)

**(2) Dataset:** The Dataset class is an abstract class in PyTorch that represents a dataset. It provides a consistent interface for accessing the training or testing data and is often used as a base class for custom dataset implementations. By inheriting from the Dataset class, you can define your own datasets and customize the data loading process. (데이터셋을 자유롭게 조작할 수 있게 한다.)

**(3) DataLoader:** The DataLoader class is used to load data from a Dataset object. It provides an iterable over the dataset and supports various options for parallel data loading, shuffling, and batching. DataLoader makes it easier to handle large datasets and efficiently feed them to a model during training or evaluation. (모델에 데이터셋을 주입할 때, 다양한 조작을 가능하게 한다.)

**(4) RandomSampler:** The RandomSampler class is a sampling strategy used by the DataLoader to randomly sample data from a dataset. It shuffles the indices of the data samples randomly at each epoch to provide a random sampling order. Random sampling is commonly used during training to ensure diverse and unbiased mini-batches. (데이터셋에서 랜덤한 인덱스를 추출하여, 학습을 가능하게 한다.)

**(5) SequentialSampler: **The SequentialSampler class is another sampling strategy used by the DataLoader. It samples data from a dataset sequentially without shuffling. In other words, it provides the data samples in the order they appear in the dataset. Sequential sampling is often used during validation or testing to ensure consistent evaluation. (랜덤 인덱스대로 데이터를 추출해 훈련시키는 것이 아니라, 앞에서부터 순서대로 정해진 배치 크기만큼 데이터를 추출해 학습시킬 때 활용한다.)

In [7]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

*cuda* 는 엔비디아 계열 GPU를 사용할 때 필요한 것이라고 한다.

In [8]:
class YourDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text) # target text 의 길이

  def __getitem__(self, index): # 특정 index에 해당하는 source text 와 target text 추출 후 str 으로 데이터타입 변환
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    # 불필요한 공백을 제거하기 위한 목적
    # This removes any excessive whitespace and ensures that words are separated by a single space.
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long),
        'source_mask': source_mask.to(dtype=torch.long),
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

# batch_encode_plus 설명

In this line, self.tokenizer refers to an instance of a tokenizer object, which is commonly used in natural language processing tasks to tokenize text and convert it into numerical representations. **The batch_encode_plus()** method of the tokenizer is used here.

Here's what each parameter does:

[source_text]: The batch_encode_plus() method expects a list of text strings as input. In this case, source_text is wrapped in a list to make it a single-item list.
max_length: This parameter specifies the maximum length of the input sequence after tokenization. If the input sequence exceeds this length, it will be truncated. If it is shorter, it will be padded.
pad_to_max_length=True: This parameter ensures that the input sequence is padded to the maximum length specified with a padding token. Padding is required to make all input sequences in a batch of equal length.
truncation=True: If the input sequence exceeds the maximum length, this parameter allows truncating it to fit within the specified limit.
padding="max_length": This parameter specifies the padding strategy. With padding="max_length", the padding is added to the end of the sequence until it reaches the maximum length.
return_tensors='pt': This parameter indicates the desired type of the returned tensors. In this case, 'pt' specifies that the tensors should be returned as PyTorch tensors.
The output of batch_encode_plus() is stored in the source variable, which contains a dictionary-like object with the tokenized and encoded version of source_text. The tokens are converted into numerical IDs, and additional information such as attention masks may also be included.

# squeeze 설명

The source['input_ids'] represents a tensor that contains the input IDs of the tokenized source text. It is likely a tensor with shape [batch_size, sequence_length], where batch_size is the number of samples in the batch, and sequence_length is the length of the tokenized sequence.

By calling squeeze() on this tensor, any singleton dimensions (dimensions with size 1) are removed. The resulting tensor will have reduced dimensionality if there were any singleton dimensions present. If there were no singleton dimensions, the tensor remains unchanged.

텐서에서 크기가 1인 차원을 제거하는 역할, 반대로 차원을 늘리고자 하는 경우에는 unsqueeze 메소드를 사용하면 된다. 다시 본론으로 돌아와서, squeeze 를 사용하는 이유는 불필요한 차원을 제거해주기 위함이다. 혹은 연산에 있어 차원의 크기를 맞춰주어야 하는 경우 사용한다.

In [9]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0): # _(underscore) 는 의미 없는 값을 표현하는 하나의 방식 (관례), 이 반복문에서는 인덱스 값이 필요없고 의미 없어서 _처리
  # enumerate의 두번째 값은 (enumerate가 부여하는) 시작 인덱스를 의미한다.
    y = data['target_ids'].to(device, dtype = torch.long) # .to 는 GPU 디바이스로 데이터를 전송하는 것
    y_ids = y[:, :-1].contiguous() # .contiguous 는 텐서의 유연한 수정을 위한 것
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100 # 패딩된 값이 영향을 미치지 않도록 -100 처리
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    # 로그 기록
    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)

    optimizer.zero_grad() # gradient 를 0으로 초기화
    loss.backward() # gradient 계산
    optimizer.step() # 모델 파라미터 업데이트

These lines extract the relevant tensors from the data dictionary. It assumes that the data dictionary contains keys such as 'target_ids', 'source_ids', and 'source_mask', representing the target labels, input IDs, and attention masks, respectively. The tensors are then moved to the specified device (likely a GPU) and cast to the torch.long data type.

In addition, y_ids is created by slicing the y tensor to exclude the last token, and lm_labels is created by cloning the sliced y tensor and detaching it from the computation graph. Further, any token in lm_labels that corresponds to the padding token (identified by tokenizer.pad_token_id) is set to a value of -100. This is typically done to ignore the loss contribution from the padding tokens during training.

In [10]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad(): # 평가 동안에는 gradient 가 업데이트 되지 않도록
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          # 디코더에서 generate 하는 옵션
          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask,
              max_length=150, # 생성 문장의 최대 길이
              num_beams=2, # beam-search 횟수
              repetition_penalty=2.5, # 반복해서 등장한 토큰에 대해 주입하는 페널티 값
              length_penalty=1.0, # 길이에 다른 페널티 값
              early_stopping=True
              )

          # skip_special_tokens: EOS 혹은 padding 같은 값 skip
          # clean_up_tokenization_space = 토큰 분리되면서 발생한 공백 제거
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]

          # 로그 기록
          if _%10==0:
              console.print(f'Completed {_}')

          # 리스트에 항목 추가 (append 와 유사)
          predictions.extend(preds)
          actuals.extend(target)

  return predictions, actuals

# generated_ids 에서 각 파라미터에 대한 추가 설명

num_beams: This parameter controls the number of beams used in beam search. Beam search is a technique used to explore multiple possible sequences during generation. Each beam represents a different sequence, and num_beams determines how many sequences are considered in parallel. In the code snippet, num_beams is set to 2, meaning that two beams will be used for generating sequences.

repetition_penalty: This parameter introduces a penalty for repeating tokens in the generated sequences. It discourages the model from generating the same token multiple times consecutively. A higher value for repetition_penalty results in more diverse and less repetitive output. In the code snippet, repetition_penalty is set to 2.5.

length_penalty: This parameter applies a penalty based on the length of the generated sequences. It balances the effect of length on the generation process. A higher value for length_penalty encourages the model to generate longer sequences, while a lower value encourages shorter sequences. In the code snippet, length_penalty is set to 1.0, which implies no additional length penalty. (일반적으로 기준이 되는 값 1.0)

These parameters allow you to control various aspects of the generation process to achieve desired output characteristics. By adjusting max_length, you can control the length of the generated sequences. num_beams enables exploration of multiple sequence possibilities. repetition_penalty penalizes repetitive tokens, and length_penalty influences the length of the generated sequences. These parameters can be tuned based on the requirements and constraints of your specific generation task.

In [11]:
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./outputs/" ):

  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)

  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]
  display_df(dataframe.head(2))


  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation.
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session.
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')

  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)

  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))

  console.save_text(os.path.join(output_dir,'logs.txt'))

  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

# Reproducibility

These lines set the random seeds and ensure deterministic behavior for reproducibility. The model_params parameter is expected to be a dictionary containing various model-related parameters, and model_params["SEED"] is used to access the seed value.

torch.manual_seed() sets the random seed for PyTorch to ensure that random processes within PyTorch are reproducible.
np.random.seed() sets the random seed for NumPy to ensure reproducibility in any NumPy operations.
torch.backends.cudnn.deterministic = True sets the deterministic property of the cuDNN backend in PyTorch to True. This ensures deterministic behavior for CUDA operations and may improve reproducibility on GPU devices.
By setting the random seeds and enabling deterministic behavior, the intention is to ensure that running the code multiple times with the same parameters and data produces the same results, making the training process reproducible.

# Data Loader 를 활용하는 이유 (df 자체가 아니라)

Batch Processing: Data loaders allow you to efficiently process data in batches during training and validation. They handle the process of dividing the dataset into batches, shuffling the data (if needed), and loading the data in parallel using multiple workers. This can significantly speed up the training process, especially when working with large datasets.

Data Transformation: Data loaders provide the flexibility to apply data transformations and preprocessing steps to the dataset on-the-fly. For example, you can apply tokenization, normalization, or data augmentation techniques within the data loader itself, without modifying the original dataframe. This allows for cleaner and more modular code organization.

Data Shuffling: During training, it is common to shuffle the data between epochs to introduce randomness and prevent the model from learning the order of the data. Data loaders can handle the shuffling of data automatically, ensuring that the model sees the data in a different order during each epoch. This is important for achieving better generalization and reducing the impact of any inherent data ordering biases.

Memory Efficiency: Data loaders can efficiently load and unload data in batches, which helps in managing memory resources effectively. They load only the necessary data for the current batch, rather than loading the entire dataset into memory. This is particularly useful when working with large datasets that may not fit entirely in memory.

Convenience and Compatibility: Data loaders provide a convenient interface for working with data in training and validation loops. They allow you to easily iterate over the dataset, access the input features and target labels, and handle batching and parallel loading seamlessly. Additionally, data loaders are compatible with various PyTorch utilities, such as distributed training and gradient accumulation.

In [12]:
model_params={
    "MODEL":"t5-base",             # model_type: t5-base/t5-large (안타깝게도 추가 결제 없이 t5-large 모델은 돌아가지 않는다.)
    "TRAIN_BATCH_SIZE":8,          # training batch size
    "VALID_BATCH_SIZE":8,          # validation batch size
    "TRAIN_EPOCHS":3,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH":50,   # max length of target text
    "SEED": 42                     # set seed for reproducibility

}

In [13]:
T5Trainer(dataframe=df[:500], source_text="text", target_text="headlines", model_params=model_params, output_dir="outputs")

[12:11:44] [Model]: Loading t5-base...                                           <ipython-input-11-5418bbc01c46>:14
                                                                                                                   

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


[12:12:01] [Data]: Reading data...                                               <ipython-input-11-5418bbc01c46>:25
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                      source_text                       |                       target_text                      |
|--------------------------------------------------------+--------------------------------------------------------|
|   summarize: Saurav Kant, an alumnus of upGrad and     |  upGrad learner switches to career in ML & Al with 90% |
|IIIT-B's PG Program in Machine learning and Artificial  |                       salary hike                      |
|Intelligence, was a Sr Systems Engineer at Infosys with |                                                        |
|  almost 5 years of work experience. The program and    |                                                        |
|     upGrad's 360-degree career support helped him      |                                                        |
| transition to a Data Scientist at Tech Mahindra with   |                                                        |
|  90% salary hike. upGrad's Online Power Learning has   |                                                        |
|               powered 3 lakh+ careers.                 |                                                        |
|   summarize: Kunal Shah's credit card bill payment     | Delhi techie wins free food from Swiggy for one year on|
| platform, CRED, gave users a chance to win free food   |                          CRED                          |
|   from Swiggy for one year. Pranav Kaushik, a Delhi    |                                                        |
|  techie, bagged this reward after spending 2000 CRED   |                                                        |
|coins. Users get one CRED coin per rupee of bill paid,  |                                                        |
|  which can be used to avail rewards from brands like   |                                                        |
|    Ixigo, BookMyShow, UberEats, Cult.Fit and more.     |                                                        |
+-----------------------------------------------------------------------------------------------------------------+

FULL Dataset: (500, 2)

...

Completed 10

[12:15:18] [Validation Completed.]                                               <ipython-input-11-5418bbc01c46>:95
                                                                                                                   

[Model] Model saved @ outputs/model_files

[Validation] Generation on Validation data saved @ outputs/predictions.csv

[Logs] Logs saved @ outputs/logs.txt